In [4]:
#Import packages

#Rdkit
from rdkit import Chem

#Scipy
from scipy import stats 
from scipy.stats import uniform,ranksums 

#Pandas
import pandas as pd
from pandas import DataFrame

#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc

#Others
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import joblib #jbolib模块
import random
import numpy as np
import os
import math
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [5]:
import torch
import deepchem as dc
import numpy as np

In [6]:
def print_RxC(dataframe):
    rows= dataframe.shape[0]
    columns= dataframe.shape[1]
    rows_columns=print(dataframe.shape[0],"Moleculars ×",dataframe.shape[1],"Descriptors")
    return rows_columns
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='spearman')
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

In [7]:
#Setting the Work Path
warnings.filterwarnings ('ignore')
#Work Path
os.chdir("C:\\Users\\A\\Desktop\\Bioactive\\IL-1β")

In [7]:
df = pd.read_csv('./AID_743279_datatable_all.csv',index_col=0)
df.dropna(subset=["PUBCHEM_CID"], inplace=True)  
df['Activity']=np.where(df['PUBCHEM_ACTIVITY_OUTCOME']=='Active', 1, 0)
df.sort_values(by="Activity" , inplace=True, ascending=False) 
df.drop_duplicates(subset='PUBCHEM_CID', keep='first', inplace=True)
df['cid']=df['PUBCHEM_CID']
df.index=df['cid']
df=df.drop(['PUBCHEM_CID'], axis=1)
df=df.drop(['PUBCHEM_SID'], axis=1)
df=df.drop(['cid'], axis=1)
df

,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,Phenotype,Potency,Efficacy,Analysis Comment,inflammasome-p1_alphalisa-Activity_Score,...,inflammasome-p1_alphalisa-Fit_R2,inflammasome-p1_alphalisa-Fit_InfiniteActivity,inflammasome-p1_alphalisa-Fit_ZeroActivity,inflammasome-p1_alphalisa-Fit_CurveClass,inflammasome-p1_alphalisa-Excluded_Points,inflammasome-p1_alphalisa-Max_Response,inflammasome-p1_alphalisa-Activity at 11.50 uM,inflammasome-p1_alphalisa-Activity at 57.50 uM,Compound QC,Activity
cid,,,,,,,,,,,,,,,,,,,,,
53300413.0,CCOC1=CC=C(C=C1)CCN2[C@H](CN(C(=O)C2=O)[C@@H](...,Active,95.0,http://assay.nih.gov/htsws/rest/display/inflam...,NaN,Inhibitor,NaN,NaN,NaN,95,...,NaN,NaN,NaN,NaN,NaN,-53.7005,-17.008,-53.7005,QC'd by Richard A Houghten - Torrey Pines Inst...,1
3247106.0,COC1=CC=C(C=C1)C[C@@]2([C@@H]3[C@@H](CC4=C3C=C...,Active,95.0,http://assay.nih.gov/htsws/rest/display/inflam...,NaN,Inhibitor,NaN,NaN,NaN,95,...,NaN,NaN,NaN,NaN,NaN,-65.634,11.1903,-65.634,QC'd by Peter Wipf - Univ. of Pittsburgh - MLI...,1
51360471.0,CCCCN1C(=NN=N1)C(C2=CC=CC3=CC=CC=C32)N4CCN(CC4...,Active,95.0,http://assay.nih.gov/htsws/rest/display/inflam...,NaN,Inhibitor,NaN,NaN,NaN,95,...,NaN,NaN,NaN,NaN,NaN,-58.6198,-8.942,-58.6198,QC'd by Chris Hulme - Univ. of Arizona - MLI PSL,1
1479279.0,CC1=CC=C(C=C1)CN2C=CC(=O)C(=C2C)OCC3=C(C=CC=C3...,Active,95.0,http://assay.nih.gov/htsws/rest/display/inflam...,NaN,Inhibitor,NaN,NaN,NaN,95,...,NaN,NaN,NaN,NaN,NaN,-58.6422,8.2068,-58.6422,QC'd by Key Organics Ltd.,1
49795501.0,CC1=C(C=C(C=C1)F)C(=O)N2CCCCC2C3=CC(=NO3)C(=O)...,Active,95.0,http://assay.nih.gov/htsws/rest/display/inflam...,NaN,Inhibitor,NaN,NaN,NaN,95,...,NaN,NaN,NaN,NaN,NaN,-55.6679,-48.1137,-55.6679,QC'd by Evotec San Francisco,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53301550.0,C[C@@H](C1=CC=CC=C1)N2C[C@@H](OC3=C(S2(=O)=O)C...,Inactive,0.0,http://assay.nih.gov/htsws/rest/display/inflam...,NaN,Inactive,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,16.6937,14.4319,16.6937,QC'd by Jeff Aube - Univ. of Kansas - MLI CMLD,0
53301551.0,C[C@@H](C1=CC=CC=C1)N2C[C@H](OC3=C(S2(=O)=O)C=...,Inactive,0.0,http://assay.nih.gov/htsws/rest/display/inflam...,NaN,Inactive,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,17.396,30.2369,17.396,QC'd by Jeff Aube - Univ. of Kansas - MLI CMLD,0
53301552.0,CCCCC[C@@H](C)N1C[C@@H](OC2=C(S1(=O)=O)C=CC(=C...,Inactive,0.0,http://assay.nih.gov/htsws/rest/display/inflam...,NaN,Inactive,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,-12.9119,9.6931,-12.9119,QC'd by Jeff Aube - Univ. of Kansas - MLI CMLD,0


In [8]:
df['IsomericSMILES']=df['PUBCHEM_EXT_DATASOURCE_SMILES']
Original_data=pd.DataFrame(df['IsomericSMILES'])
Original_data['Activity']=df['Activity']
Chemdraw_smiles=Original_data['IsomericSMILES'].tolist()

Original_data

,IsomericSMILES,Activity
cid,,
53300413.0,CCOC1=CC=C(C=C1)CCN2[C@H](CN(C(=O)C2=O)[C@@H](...,1
3247106.0,COC1=CC=C(C=C1)C[C@@]2([C@@H]3[C@@H](CC4=C3C=C...,1
51360471.0,CCCCN1C(=NN=N1)C(C2=CC=CC3=CC=CC=C32)N4CCN(CC4...,1
1479279.0,CC1=CC=C(C=C1)CN2C=CC(=O)C(=C2C)OCC3=C(C=CC=C3...,1
49795501.0,CC1=C(C=C(C=C1)F)C(=O)N2CCCCC2C3=CC(=NO3)C(=O)...,1
...,...,...
53301550.0,C[C@@H](C1=CC=CC=C1)N2C[C@@H](OC3=C(S2(=O)=O)C...,0
53301551.0,C[C@@H](C1=CC=CC=C1)N2C[C@H](OC3=C(S2(=O)=O)C=...,0
53301552.0,CCCCC[C@@H](C)N1C[C@@H](OC2=C(S1(=O)=O)C=CC(=C...,0


In [9]:
Smis=[]
for i in Chemdraw_smiles:
    Smis.append(i)
Mols = []
for Smi in Smis:
    Mol = Chem.MolFromSmiles(Smi)
    Mols.append(Mol)
Canonical_smis = []
for Mol in Mols:
    Canonical_smi = Chem.MolToSmiles(Mol)
    Canonical_smis.append(Canonical_smi)
Original_data['Canonical_smiles']=Canonical_smis
Original_data.to_csv("./Original_data.csv",sep=',',header=1,index=1)
Original_data

,IsomericSMILES,Activity,Canonical_smiles
cid,,,
53300413.0,CCOC1=CC=C(C=C1)CCN2[C@H](CN(C(=O)C2=O)[C@@H](...,1,CCOc1ccc(CCN2C(=O)C(=O)N([C@@H](CC(C)C)CN3CCC[...
3247106.0,COC1=CC=C(C=C1)C[C@@]2([C@@H]3[C@@H](CC4=C3C=C...,1,COC(=O)[C@@]1(Cc2ccc(OC)cc2)[C@H]2c3cc(C(=O)N4...
51360471.0,CCCCN1C(=NN=N1)C(C2=CC=CC3=CC=CC=C32)N4CCN(CC4...,1,CCCCn1nnnc1C(c1cccc2ccccc12)N1CCN(C/C=C/c2cccc...
1479279.0,CC1=CC=C(C=C1)CN2C=CC(=O)C(=C2C)OCC3=C(C=CC=C3...,1,Cc1ccc(Cn2ccc(=O)c(OCc3c(Cl)cccc3Cl)c2C)cc1
49795501.0,CC1=C(C=C(C=C1)F)C(=O)N2CCCCC2C3=CC(=NO3)C(=O)...,1,Cc1ccc(F)cc1C(=O)N1CCCCC1c1cc(C(=O)NC(C)C)no1
...,...,...,...
53301550.0,C[C@@H](C1=CC=CC=C1)N2C[C@@H](OC3=C(S2(=O)=O)C...,0,C[C@@H](c1ccccc1)N1C[C@H](COCc2ccccc2)Oc2cc(N3...
53301551.0,C[C@@H](C1=CC=CC=C1)N2C[C@H](OC3=C(S2(=O)=O)C=...,0,C[C@@H](c1ccccc1)N1C[C@@H](COCc2ccccc2)Oc2cc(N...
53301552.0,CCCCC[C@@H](C)N1C[C@@H](OC2=C(S1(=O)=O)C=CC(=C...,0,CCCCC[C@@H](C)N1C[C@H](COCc2ccccc2)Oc2cc(N3CC[...


In [8]:
Original_data=pd.read_csv('./data_psm.csv',index_col=0)

In [9]:
#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc
#AlvaDesc Path(Linux default alvaDescCLI location)
aDesc = AlvaDesc('E:\\C++\\alvaDesc\\alvaDescCLI.exe') 

In [11]:
#Calculate the descriotors
aDesc.set_input_SMILES(Original_data['Canonical_smiles'].tolist())
if not aDesc.calculate_descriptors('ALL'):
 print('Error: ' + aDesc.get_error())
else:
 Data_gelation=aDesc.get_output()

In [12]:
X_original_data=pd.DataFrame(Data_gelation)
Desc_names_names=aDesc.get_output_descriptors()
X_original_data.columns=Desc_names_names
X_original_data.index=Original_data.index

X_original_data.head()

,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,Mi,...,s1_numAroBonds,s2_numAroBonds,s3_numAroBonds,s4_numAroBonds,s34_size,s34_relSize,s34_phSize,s34_phRelSize,chiralMoment,chiralPhMoment
OPTUM_LAB_ID,,,,,,,,,,,,,,,,,,,,,
53300413.0,704.02,6.518704,63.0926,107.4464,67.5519,122.5431,0.584191,0.994874,0.625481,1.134658,...,0.0,0.0,6.0,6.0,46.75,0.916667,10.625,0.208333,50.709541,12.001003
49795501.0,373.47,7.322941,31.3867,51.5196,32.6935,58.0300,0.615425,1.010188,0.641049,1.137843,...,0.0,0.0,5.0,6.0,23.50,0.870370,6.000,0.222222,21.400935,6.633250
9551212.0,487.74,6.869577,41.8673,70.7049,45.2221,80.6829,0.589680,0.995844,0.636931,1.136379,...,0.0,0.0,0.0,0.0,0.00,0.000000,0.000,0.000000,0.000000,0.000000
53300217.0,515.47,8.182063,40.9322,63.5306,43.2048,70.5350,0.649717,1.008422,0.685790,1.119603,...,0.0,0.0,6.0,6.0,25.50,0.728571,4.000,0.114286,20.296826,3.602880
1254489.0,477.98,8.241034,38.5457,58.6579,40.0684,64.8437,0.664581,1.011343,0.690834,1.117995,...,0.0,0.0,0.0,0.0,0.00,0.000000,0.000,0.000000,0.000000,0.000000


In [13]:
#Remove descriptors with excessive NA values
Thresh =int(0.8*len(X_original_data))
X_NAomit_data=X_original_data.dropna(
    #thresh=Thresh, 
    axis=1) 
X_NAomit_data.head()
X_NAomit_data.to_csv("./X_NAomit_data.csv",sep=',',header=1,index=1)

In [14]:
#Ranksum test
Ranksums_data=X_NAomit_data
Ranksums_data["Activity"]=Original_data['Activity']
Notgelable=Ranksums_data.loc[Ranksums_data["Activity"] == 0]
Gelable=Ranksums_data.loc[Ranksums_data["Activity"] == 1]
Ranksums_result=[]
for i in list(Ranksums_data.columns):
 NG = Notgelable[i]
 G = Gelable[i]
 Ranksums_caculate=ranksums(NG, G, alternative='two-sided')
 Ranksums_result.append(Ranksums_caculate)
Ranksums_results = pd.DataFrame(Ranksums_result, columns=['Statistics','Pvalues'])
Ranksums_results.index=Ranksums_data.columns#
Ranksums_results = Ranksums_results.drop('Activity',axis = 0)
Ranksums_results.sort_values(by="Pvalues" , inplace=True, ascending=True)
Ranksums_results

,Statistics,Pvalues
ESOL,13.329526,1.558854e-40
LOGPcons,-13.123335,2.420455e-39
LOGP99,-12.840978,9.664605e-38
MRcons,-10.873548,1.540904e-27
MR99,-10.873548,1.540904e-27
...,...,...
B06[Cl-Cl],0.000000,1.000000e+00
B06[F-X],0.000000,1.000000e+00
B06[F-Si],0.000000,1.000000e+00
B06[F-I],0.000000,1.000000e+00


In [15]:
#Ranksum test <0.05
Ranksums_marked = Ranksums_results[Ranksums_results['Pvalues'] <0.05]
print(Ranksums_marked.shape)
Ranksums_marked.head()

(1416, 2)


,Statistics,Pvalues
ESOL,13.329526,1.558854e-40
LOGPcons,-13.123335,2.420455e-39
LOGP99,-12.840978,9.664605e-38
MRcons,-10.873548,1.540904e-27
MR99,-10.873548,1.540904e-27


In [16]:
X_ranksums_data=X_NAomit_data[Ranksums_marked.index]
X_ranksums_data.head()

,ESOL,LOGPcons,LOGP99,MRcons,MR99,SM5_B(p),SM6_B(p),SM3_B(p),SM4_B(p),H_Dz(p),...,C-006,Chi_Dz(p),Eta_D_epsiB,Eig01_AEA(ed),SpMax_AEA(ed),GATS7e,SpDiam_AEA(ri),MATS1i,F06[N-N],s3_phRelSize_2
OPTUM_LAB_ID,,,,,,,,,,,,,,,,,,,,,
53300413.0,-5.437540,3.844901,3.86770,196.143502,196.143502,8.285057,9.468507,6.020839,7.206601,208.184773,...,7.0,0.107313,0.025000,6.807807,6.807807,1.143083,5.266170,-0.221142,3.0,0.217461
49795501.0,-4.371830,3.341994,3.62772,97.870701,97.870701,7.756864,8.978804,5.420658,6.652109,89.622120,...,1.0,0.199496,0.040982,6.800146,6.800146,1.184350,5.215982,-0.089817,1.0,0.363636
9551212.0,-3.781132,2.512651,2.87710,134.857902,134.857902,7.913694,9.116032,5.634128,6.816039,115.146360,...,6.0,0.114427,0.013646,6.977538,6.977538,0.964725,5.393584,-0.194756,0.0,0.000000
53300217.0,-5.610078,4.655003,5.27560,134.407701,134.407701,8.070163,9.286880,5.742054,6.957621,143.748853,...,3.0,0.171284,0.055521,8.007991,8.007991,0.956364,5.714619,-0.083474,0.0,0.148601
1254489.0,-5.277250,3.675054,4.32590,131.395700,131.395700,8.065041,9.291004,5.726890,6.954949,128.789115,...,6.0,0.139375,0.075758,6.397235,6.397235,0.756414,5.115151,-0.191836,0.0,0.000000


In [17]:
#Scale data
Scaler = preprocessing.MinMaxScaler() #StandardScaler
Transformer =Scaler.fit(X_ranksums_data)
X_scaled_data=Transformer.transform(X_ranksums_data)
X_scaled_data =pd.DataFrame(X_scaled_data)
X_scaled_data.columns=X_ranksums_data.columns
X_scaled_data.index=Original_data.index
X_scaled_data.to_csv("./X_scaled_data.csv",sep=',',header=1,index=1)
joblib.dump(Transformer, './Scaler_transformer.pkl')
X_scaled_data.head()

,ESOL,LOGPcons,LOGP99,MRcons,MR99,SM5_B(p),SM6_B(p),SM3_B(p),SM4_B(p),H_Dz(p),...,C-006,Chi_Dz(p),Eta_D_epsiB,Eig01_AEA(ed),SpMax_AEA(ed),GATS7e,SpDiam_AEA(ri),MATS1i,F06[N-N],s3_phRelSize_2
OPTUM_LAB_ID,,,,,,,,,,,,,,,,,,,,,
53300413.0,0.374972,0.693125,0.704468,0.838102,0.838102,0.863458,0.843964,0.898451,0.885361,0.756514,...,0.411765,0.043701,0.174087,0.301063,0.301063,0.486872,0.305242,0.514046,0.3,0.217461
49795501.0,0.483465,0.655272,0.689435,0.305627,0.305627,0.631307,0.643839,0.591432,0.620820,0.294228,...,0.058824,0.164470,0.285376,0.298794,0.298794,0.506351,0.271018,0.675998,0.1,0.363636
9551212.0,0.543601,0.592850,0.642413,0.506036,0.506036,0.700237,0.699920,0.700632,0.699028,0.393749,...,0.352941,0.053021,0.095026,0.351339,0.351339,0.402679,0.392128,0.546586,0.0,0.000000
53300217.0,0.357407,0.754099,0.792663,0.503597,0.503597,0.769008,0.769739,0.755840,0.766575,0.505272,...,0.176471,0.127510,0.386620,0.656570,0.656570,0.398732,0.611050,0.683821,0.0,0.148601
1254489.0,0.391290,0.680341,0.733171,0.487277,0.487277,0.766757,0.771425,0.748083,0.765301,0.446943,...,0.352941,0.085705,0.527541,0.179448,0.179448,0.304347,0.202259,0.550187,0.0,0.000000


In [18]:
#Remove high correlation descriptors
X_cor_data=trimm_correlated(X_scaled_data,0.80)
X_cor_data.index=Original_data.index
X_cor_data.to_csv("./ML_data.csv",sep=',',header=1,index=1)
print_RxC(X_cor_data)
X_cor_data.head()

1134 Moleculars × 156 Descriptors


,ESOL,MRcons,SHED_LL,QED,MaxaaCH,SpMin1_Bh(e),SpMin2_Bh(i),SpMax1_Bh(p),Eta_D_psiA,SpMax2_Bh(p),...,Eig08_EA(dm),CATS2D_03_DA,C-003,MATS3v,B10[C-O],F09[N-N],P_VSA_charge_1,C-006,GATS7e,F06[N-N]
OPTUM_LAB_ID,,,,,,,,,,,,,,,,,,,,,
53300413.0,0.374972,0.838102,0.947371,0.382994,0.891586,0.639835,0.779939,0.572376,0.414202,0.762623,...,0.549857,0.222222,0.333333,0.499714,1.0,0.4,0.204474,0.411765,0.486872,0.3
49795501.0,0.483465,0.305627,0.613480,0.927428,0.705690,0.655720,0.610128,0.556341,0.543594,0.658486,...,0.317460,0.111111,0.000000,0.588836,1.0,0.0,0.077861,0.058824,0.506351,0.1
9551212.0,0.543601,0.506036,0.843188,0.561356,0.000000,0.513069,0.654981,0.782782,0.384274,0.622166,...,0.407043,0.111111,0.166667,0.602432,1.0,0.2,0.098737,0.352941,0.402679,0.0
53300217.0,0.357407,0.503597,0.733585,0.562062,0.848486,0.637951,0.755576,0.647745,0.391578,0.752076,...,0.317460,0.000000,0.166667,0.498670,1.0,0.0,0.081053,0.176471,0.398732,0.0
1254489.0,0.391290,0.487277,0.825753,0.621663,0.873455,0.663602,0.765541,0.572797,0.422338,0.763210,...,0.317460,0.000000,0.000000,0.553357,1.0,0.0,0.116679,0.352941,0.304347,0.0
